# Introduction

Name: Melissa Sanchez

I understand that my submission needs to be my own work: YES

I understand that ChatGPT / Copilot / other AI tools are not allowed: MS

Total Points: **75**

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [2]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 18.2 MB/s eta 0:00:00


In [2]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(3 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(3 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(4 point)**

Solution:

1) 0 <= j < j' <= 8     ¬( X(k=5 ,i=0 ,j) 𝝠  X(k=5 ,i=0 ,j'))

2) 0 <=i < i' <= 8      ¬( X(k=5 ,i ,j=0) 𝝠  X(k=5 ,i' ,j=0))

3) 0 <= i, i' <= 2, 0 <= j, j' <= 2, (i,j) != (i',j')      ¬( X(k=9 ,i ,j) 𝝠 X(k=9 ,i' ,j'))


## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [1]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *


s = Solver()

#k=number,i=row,j=column
board = [[[ Bool("X,%d,%d,%d" % (k+1, i, j)) for k in range(9) ]
       for j in range(9)]
     for i in range(9)]

#constraint: one digit per cell
for i in range(9):
    for j in range(9):
        s.add(Or([ board[k][i][j] for k in range(9) ]))


        #only one disig can be in the cell
        #1<=n<m<=9
        for n in range(9):
          for m in range(n+1,9):
            s.add(Not(And(board[n][i][j], board[m][i][j])))

#constraint: non-repeating numbers in each row
for i in range(9):
  for k in range(9):
    for j in range(9):
      for d in range(j+1,9):
        s.add(Not(And(board[k][i][j],board[k][i][d])))

#constraint: non-repeating numbers in each column
for j in range(9):
  for k in range(9):
    for i in range(9):
      for d in range(i+1,9):
        s.add(Not(And(board[k][i][j],board[k][d][j])))

#constraint: non-repeating numbers in each 3x3 subgrid
for block_i in range(3):
  for block_j in range(3):
    #bread down into 6 sections 3 by 3
    # 00 01 02
    # 10 11 12 -------> first block 00 ----> 00 01 02
    # 20 21 22                               10 11 12
    #                                        20 21 22
    #get all coordinages of each section
    cells = [(block_i * 3 + i, block_j * 3 + j) for i in range(3) for j in range(3)]
    for k in range(9):
      for idx in range(len(cells)):
                for jdx in range(idx+1, len(cells)):
                  #ensure non-repeating
                    i1, j1 = cells[idx]
                    i2, j2 = cells[jdx]
                    s.add(Not(And(board[k][i1][j1], board[k][i2][j2])))





### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(5 points)**

In [5]:
sudoku_board = [[2, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 7, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 6, 0],
                [0, 0, 0, 0, 0, 0, 3, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 4, 0, 0, 0, 0, 0, 0, 6],
                [0, 0, 0, 0, 0, 2, 0, 0, 0],
                [0, 0, 0, 0, 0, 9, 0, 0, 0],
                [0, 0, 5, 0, 8, 0, 0, 0, 0]]
#sudoku_board = [[0 for j in range(9)] for i in range(9)]
#add the pre-existing values of the sudoku board as a constraint
for i in range(9):
    for j in range(9):
        if sudoku_board[i][j] != 0:
           k = sudoku_board[i][j]
           s.add(board[k -1][i][j])

#check for a solution
if s.check() == sat:
  m = s.model()
  #create another sudoku board for solution visualization
  sol = [[0 for j in range(9)] for i in range(9)]

  #get the 'solution' to the sudoku
  for i in range(9):
        for j in range(9):
            for k in range(9):
                if m.evaluate(board[k][i][j]):
                    sol[i][j] = k + 1
  #print the solution

  for i in range(9):
      if i % 3 == 0 and i != 0:
          print("-" * 21)  #divive each 3x3 subgrid
      row = ""
      for j in range(9):
          if j % 3 == 0 and j != 0:
              row += " | " #divive each 3x3 subgrid
          row += " " + str(sol[i][j])
      print(row)

else:
  print("unsat")


 2 8 1 |  3 9 6 |  7 4 5
 6 5 4 |  7 2 8 |  1 3 9
 3 9 7 |  5 4 1 |  8 6 2
---------------------
 5 2 6 |  9 1 4 |  3 7 8
 1 3 8 |  2 6 7 |  5 9 4
 7 4 9 |  8 3 5 |  2 1 6
---------------------
 4 6 3 |  1 5 2 |  9 8 7
 8 1 2 |  4 7 9 |  6 5 3
 9 7 5 |  6 8 3 |  4 2 1


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be fewer than 5000. **(5 points)**

In [2]:
sudoku_board = [[2, 3, 7, 1, 9, 4, 8, 5, 6],
                [4, 0, 6, 7, 2, 0, 0, 0, 0],
                [5, 8, 9, 0, 4, 0, 0, 6, 0],
                [0, 0, 0, 0, 5, 0, 3, 0, 0],
                [0, 0, 0, 0, 3, 0, 0, 0, 0],
                [0, 4, 0, 0, 7, 0, 0, 0, 6],
                [0, 0, 0, 0, 6, 2, 0, 0, 0],
                [0, 0, 0, 0, 1, 9, 0, 0, 0],
                [0, 0, 5, 0, 8, 0, 0, 0, 0]]

solution_count = 0
solutions = []

while s.check() == sat:
  solution_count += 1
  m = s.model()

  sol = [[0 for _ in range(9)] for _ in range(9)]
  block_sol= []
  for i in range(9):
      for j in range(9):
          for k in range(9):
              if m.evaluate(board[k][i][j]):
                  sol[i][j] = d + 1
                  # To block this solution, note the digit chosen.
                  block_sol.append(board[k][i][j])
                  break

  #print solution.
  # for row in sol:
  #     print(row)
  # print()

  #block this solution from appearing again
  s.add(Not(And(block_sol)))
print("Unique solutions:", solution_count)

Unique solutions: 4452


# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [4]:
# Solution code for Miracle Sudoku using z3py
from z3 import *
s = Solver()

#k=number,i=row,j=column
board = [[[ Bool("X,%d,%d,%d" % (k+1, i, j)) for k in range(9) ]
       for j in range(9)]
     for i in range(9)]

#regular sodoku rules
#constraint: one digit per cell
for i in range(9):
    for j in range(9):
        s.add(Or([ board[k][i][j] for k in range(9) ]))


        #only one disig can be in the cell
        #1<=n<m<=9
        for n in range(9):
          for m in range(n+1,9):
            s.add(Not(And(board[n][i][j], board[m][i][j])))

#constraint: non-repeating numbers in each row
for i in range(9):
  for k in range(9):
    for j in range(9):
      for d in range(j+1,9):
        s.add(Not(And(board[k][i][j],board[k][i][d])))

#constraint: non-repeating numbers in each column
for j in range(9):
  for k in range(9):
    for i in range(9):
      for d in range(i+1,9):
        s.add(Not(And(board[k][i][j],board[k][d][j])))

#constraint: non-repeating numbers in each 3x3 subgrid
for block_i in range(3):
  for block_j in range(3):
    #bread down into 6 sections 3 by 3
    # 00 01 02
    # 10 11 12 -------> first block 00 ----> 00 01 02
    # 20 21 22                               10 11 12
    #                                        20 21 22
    #get all coordinages of each section
    cells = [(block_i * 3 + i, block_j * 3 + j) for i in range(3) for j in range(3)]
    for k in range(9):
      for idx in range(len(cells)):
                for jdx in range(idx+1, len(cells)):
                  #ensure non-repeating
                    i1, j1 = cells[idx]
                    i2, j2 = cells[jdx]
                    s.add(Not(And(board[k][i1][j1], board[k][i2][j2])))
#king's move
#(di,dj in {-1,0,1} except (0,0)).
for i in range(9):
    for j in range(9):
        for di in range(-1, 2):
            for dj in range(-1, 2):
                if di == 0 and dj == 0:
                    continue
                ni, nj = i + di, j + dj
                if 0 <= ni < 9 and 0 <= nj < 9:
                    for k in range(9):
                        s.add(Not(And(board[k][i][j], board[k][ni][nj])))

#knight's move
#|di|*|dj| == 2.
for i in range(9):
    for j in range(9):
        for di in range(-2, 3):  # di -> {-2, -1, 0, 1, 2}
            for dj in range(-2, 3):  # dj -> {-2, -1, 0, 1, 2}
                if abs(di) * abs(dj) == 2:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < 9 and 0 <= nj < 9:
                        for k in range(9):
                            s.add(Not(And(board[k][i][j], board[k][ni][nj])))

#orthogonally adjacent
for i in range(9):
    for j in range(9):
        for (di, dj) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < 9 and 0 <= nj < 9:
                #for any digit is not possible to have processors or predessesors
                for k in range(9):
                    if k > 0:
                        s.add(Not(And(board[k][i][j], board[k-1][ni][nj])))
                    if k < 8:
                        s.add(Not(And(board[k][i][j], board[k+1][ni][nj])))

sudoku_board = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 2, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0]]
#add the board pre-existing entries
for i in range(9):
    for j in range(9):
        if sudoku_board[i][j] != 0:
            k = sudoku_board[i][j]
            s.add(board[k -1][i][j])

if s.check() == sat:
    m = s.model()

    sol = [[0 for _ in range(9)] for _ in range(9)]

    for i in range(9):
        for j in range(9):
            for k in range(9):
                if m.evaluate(board[k][i][j]):
                    sol[i][j] = k + 1
                    break


    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 25)
        row_str = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row_str += " |"
            row_str += " " + str(sol[i][j])
        print(row_str)
else:
    print("unsat")


 4 8 3 | 7 2 6 | 1 5 9
 7 2 6 | 1 5 9 | 4 8 3
 1 5 9 | 4 8 3 | 7 2 6
-------------------------
 8 3 7 | 2 6 1 | 5 9 4
 2 6 1 | 5 9 4 | 8 3 7
 5 9 4 | 8 3 7 | 2 6 1
-------------------------
 3 7 2 | 6 1 5 | 9 4 8
 6 1 5 | 9 4 8 | 3 7 2
 9 4 8 | 3 7 2 | 6 1 5


---
# Number Challenge in Z3py

Write Z3py code to solve the Number Challenge puzzle shown below and print the solution found. You can use the `Int` type to create integer variables in z3py. **(20 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/chal.jpg" width="300"/>

In [22]:
# Write your solution code for Number Challenge using z3py
from z3 import *


s = Solver()


grid = [Int(f'{i}') for i in range(8)]

#add all the possible number options
for num in grid:
  s.add(num >= 1, num <= 8)

#ensure every pair of values is different
for i in range(8):
    for j in range(i+1,8):
        s.add(grid[i] != grid[j])

pos =      [(0,1), (0,2),
     (1,0), (1,1), (1,2), (1,3),
            (2,1), (2,2)]

#consecutive numbers and diagonal consecutive
for i in range(8):
  for j in range(i+1,8):
    r1, c1 = pos[i]
    r2, c2 = pos[j]
    if abs(r1 - r2) <= 1 and abs(c1 - c2) <= 1:
      s.add(Abs(grid[i] - grid[j]) != 1)

if s.check() == sat:
  m = s.model()
  sol = [ m[c].as_long() for c in grid ]

  print(f'   |{sol[0]} | {sol[1]}|')
  print("-------------")
  print(f'{sol[2]} | {sol[3]} | {sol[4]} | {sol[5]}')
  print("-------------")
  print(f'   |{sol[6]} | {sol[7]}|')
else:
  print('unsat')


   |5 | 3|
-------------
2 | 8 | 1 | 7
-------------
   |6 | 4|





---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(7 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

p  q  r  s  ¬p  t  ¬s | p -> q | (p -> q) -> r | s -> ¬p | t -> q
T  T  T  F   F  T   T      T          T           T           T
F  T  T  T   T  F   F      T          T           T           T
F  F  T  T   T  F   F      T          T           T           T




p q | q -> p| p -> (q -> p)| p -> (p -> (q -> p)) | q -> (p -> (p -> (q -> p)) )
T T     T           T               T                        T
T F     T           T               T                        T
F F     T           T               T                        T
F T     F           T               T                        T



---


 ## Logical Entailment using z3py

 Use Z3py to prove the above two entailments are true using satisfiability. See section 7.5 in the book for details. **(8 points)**

In [35]:
from z3 import *

s1 = Solver()

p, q, r, s, t = Bools('p q r s t')

s1.add(Implies(Implies(p,q),r)) # (p -> q) -> r
s1.add(Implies(s,Not(p))) # s -> ! p
s1.add(t)
s1.add(Not(s))
s1.add(Implies(t,q)) # t -> q
s1.add(Not(r)) #negation of solution

if s1.check() == sat:
  print('invalid')
else:
  print('valid')


p, q = Bools('p q')
s2 = Solver()
proposition = Implies(q,Implies(p,Implies(p,Implies(q,p))))
s2.add(Not(proposition))

if s2.check() == sat:
  print('invalid')
else:
  print('valid')

valid
valid
